In [ ]:
import ct.mlexp2 as ml
from importlib import reload
from datetime import date
from cdcqr.common.utils import load_df, save, resample_pv
from cdcqr.data.glassnode.utils import get_glassnode_data, quantile_info, url_parser
import time
from IPython.display import Audio
from cdcqr.common.config import GLASSNODE_API_KEY
sound_file = '/tmp/ding.wav'
import sys
sys.path.append('/github/cryptoderiv-quant/')
from ct.utils import glassnode


### prepare data

In [ ]:
data_sym = 'BTCUSD_PERP@binance'
df1 = load_df(data_sym).resample_pv('10Min')
df1.head()

In [ ]:
 dfgsi

In [ ]:
category = 'Transactions'
transactions_features_exchange_activity = ['TransfersVolumeToExchangesSum',
                                             'TransfersVolumeFromExchangesSum',
                                             'TransfersVolumeExchangesNet',
                                             'TransfersToExchangesCount',
                                             'TransfersFromExchangesCount',
                                             'TransfersVolumeToExchangesMean',
                                             'TransfersVolumeFromExchangesMean',
                                             'TransfersVolumeWithinExchangesSum',
                                             'TransfersBetweenExchangesCount',
                                             'TransfersVolumeBetweenExchangesSum']

for feature_name in transactions_features_exchange_activity:
    print('------------{}----------------'.format(feature_name))
    for underly in ['BTC']:
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        start_time = df1.index[0].isoformat()
        url1 = 'metrics~{}~{}'.format(category, feature)
        print(url1)
        dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        df1 = df1[df1.index<='20211031']
        dff = dfgsi.reindex(df1.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        display(dff.head())
        res = ml.mlexp2(dff, df1, sym=sym,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt','vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)
        Audio(sound_file, autoplay=True)

In [ ]:
for feature_name in ['TransfersVolumeToExchangesSum',
                                             'TransfersVolumeFromExchangesSum',
                                             'TransfersVolumeExchangesNet',
                                             'TransfersToExchangesCount',
                                             'TransfersFromExchangesCount',
                                             'TransfersVolumeToExchangesMean',
                                             'TransfersVolumeFromExchangesMean',
                                             'TransfersVolumeWithinExchangesSum',
                                             'TransfersBetweenExchangesCount',
                                             'TransfersVolumeBetweenExchangesSum']: #= 'transfers_volume_to_exchanges_mean'

    category = 'transactions'

    sym = 'BTC'
    url1 = 'metrics~{}~{}'.format(category, feature_name)
    freq='10m'
    start_time = df1.index[0].isoformat()

    dfgsi=glassnode(url1,a=sym,i=freq,s=start_time,price=False)
    dfgsi.head()

    df1 = df1[df1.index<='20211031']
    dff = dfgsi.reindex(df1.index)

    reload(ml)
    t0 = time.time()

    res = ml.mlexp2(dff, df1, sym=sym,
          qsuffix='USDT@binancealt',
          ytrain_thresh=6,
          ytrain_periods=100,
          ytest_thresh=3,
          ytest_periods=100,
          vdelay=2,
          ytypes=["ys"],
          xtypes=["v"],
          qlags=1000,
          request=['vbt','vbtplot','plottree','showsplit'],
          #models=["oneiflarger","dt"],
          models=["oneiflarger", 'rf'],
          randcv_n_iter=30,
          sl=2,
          tp=6,
          ts=None,
          vbtfreq="10Min") 
    t1 = time.time()
    print(t1-t0)
    Audio(sound_file, autoplay=True)